In [168]:
from pathlib import Path 
import requests
#import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime 
from datetime import date,datetime
import pandas as pd



In [169]:

sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483*",
    security_token="1B6xNEvzIe5uroF6nRgKGTSk",
)

In [170]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [171]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(7394, 8)

In [172]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7093, 5)

In [173]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7400, 10)

fin empresas y comienzo de rescate de pacientes


In [174]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1787077, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [175]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1381511, 4)

In [176]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1381511, 13)

In [177]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [178]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [179]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [180]:
#pacientes_empresas22.shape

In [181]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [182]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [183]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [201]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "THB Seguros"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [202]:
Pacientes_Archivo.head(10)

,poliza,GRUPO,NOMBREGRUPO,RUTASEGURADO,DV,CARGA,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,nacimiento,sexo,direccion,comuna,ciudad,celular,NOMBRE,apellidos,nombres,email
0,340023981,1,Asegurados,9462157,7,0,9462157,7,9462157-7,1972-03-07,F,NaN,NaN,NaN,22760503,Aranda Ortega Claudia,Aranda,Ortega Claudia,CLAUDIA.ARANDA@COGNITA.COM
1,340023981,1,Asegurados,9462157,7,2,20431786,0,20431786-0,2000-09-25,F,NaN,NaN,NaN,22760503,Alvarez Aranda M Ignacia,Alvarez,Aranda M Ignacia,CLAUDIA.ARANDA@COGNITA.COM
2,340023981,1,Asegurados,9462157,7,3,21182255,4,21182255-4,2003-09-08,F,NaN,NaN,NaN,22760503,Alvarez Aranda Antonia,Alvarez,Aranda Antonia,CLAUDIA.ARANDA@COGNITA.COM
3,340023981,1,Asegurados,12166073,3,0,12166073,3,12166073-3,1979-01-13,M,RULO SECTOR RURAL SIN NUMERO RULO,CALBUCO,NaN,NaN,Cruz Fuentealba Eduardo Andres,Cruz,Fuentealba Eduardo Andres,NaN
4,340023981,1,Asegurados,13060950,3,0,13060950,3,13060950-3,1976-07-14,M,NaN,NaN,NaN,NaN,Prat Ayala Rodrigo Antonio,Prat,Ayala Rodrigo Antonio,rodrigopratayala@gmail.com
5,340023981,1,Asegurados,13435972,2,0,13435972,2,13435972-2,1978-11-23,M,LOS CARPINTEROS NO 10031,LAS CONDES,SANTIAGO,942621442,Soler Rodríguez Antonio,Soler,Rodríguez Antonio,antonio.soler@innovationhealth.cl
6,340023981,1,Asegurados,13435972,2,1,22377618,3,22377618-3,2007-04-18,F,LOS CARPINTEROS NO 10031,LAS CONDES,SANTIAGO,942621442,Soler Baraona Dominga,Soler,Baraona Dominga,antonio.soler@innovationhealth.cl
7,340023981,1,Asegurados,13435972,2,2,22996636,7,22996636-7,2009-04-13,M,LOS CARPINTEROS NO 10031,LAS CONDES,SANTIAGO,942621442,Soler Baraona Clemente,Soler,Baraona Clemente,antonio.soler@innovationhealth.cl
8,340023981,1,Asegurados,13435972,2,3,24587377,8,24587377-8,2014-04-07,F,LOS CARPINTEROS NO 10031,LAS CONDES,SANTIAGO,942621442,Soler Baraona Laura,Soler,Baraona Laura,antonio.soler@innovationhealth.cl
9,340023981,1,Asegurados,13435972,2,4,13550868,3,13550868-3,1979-09-12,F,LOS CARPINTEROS NO 10031,LAS CONDES,SANTIAGO,942621442,Baraona Menchaca Magdalena,Baraona,Menchaca Magdalena,antonio.soler@innovationhealth.cl


In [203]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [204]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,poliza,GRUPO,NOMBREGRUPO,RUTASEGURADO,DV,CARGA,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,nacimiento,sexo,direccion,comuna,ciudad,celular,NOMBRE,apellidos,nombres,email,rut
0,340023981,1,Asegurados,9462157,7,0,9462157,7,9462157-7,1972-03-07,F,NaN,NaN,NaN,22760503,Aranda Ortega Claudia,Aranda,Ortega Claudia,CLAUDIA.ARANDA@COGNITA.COM,94621577
1,340023981,1,Asegurados,9462157,7,2,20431786,0,20431786-0,2000-09-25,F,NaN,NaN,NaN,22760503,Alvarez Aranda M Ignacia,Alvarez,Aranda M Ignacia,CLAUDIA.ARANDA@COGNITA.COM,204317860
2,340023981,1,Asegurados,9462157,7,3,21182255,4,21182255-4,2003-09-08,F,NaN,NaN,NaN,22760503,Alvarez Aranda Antonia,Alvarez,Aranda Antonia,CLAUDIA.ARANDA@COGNITA.COM,211822554
3,340023981,1,Asegurados,12166073,3,0,12166073,3,12166073-3,1979-01-13,M,RULO SECTOR RURAL SIN NUMERO RULO,CALBUCO,NaN,NaN,Cruz Fuentealba Eduardo Andres,Cruz,Fuentealba Eduardo Andres,NaN,121660733
4,340023981,1,Asegurados,13060950,3,0,13060950,3,13060950-3,1976-07-14,M,NaN,NaN,NaN,NaN,Prat Ayala Rodrigo Antonio,Prat,Ayala Rodrigo Antonio,rodrigopratayala@gmail.com,130609503


In [205]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_13456\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,poliza,GRUPO,NOMBREGRUPO,RUTASEGURADO,DV,CARGA,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,nacimiento,sexo,direccion,comuna,ciudad,celular,NOMBRE,apellidos,nombres,email,rut
0,340023981,1,Asegurados,9462157,7,0,9462157,7,9462157-7,1972-03-07,F,NaN,NaN,NaN,22760503,Aranda Ortega Claudia,Aranda,Ortega Claudia,CLAUDIA.ARANDA@COGNITA.COM,94621577
1,340023981,1,Asegurados,9462157,7,2,20431786,0,20431786-0,2000-09-25,F,NaN,NaN,NaN,22760503,Alvarez Aranda M Ignacia,Alvarez,Aranda M Ignacia,CLAUDIA.ARANDA@COGNITA.COM,204317860
2,340023981,1,Asegurados,9462157,7,3,21182255,4,21182255-4,2003-09-08,F,NaN,NaN,NaN,22760503,Alvarez Aranda Antonia,Alvarez,Aranda Antonia,CLAUDIA.ARANDA@COGNITA.COM,211822554
3,340023981,1,Asegurados,12166073,3,0,12166073,3,12166073-3,1979-01-13,M,RULO SECTOR RURAL SIN NUMERO RULO,CALBUCO,NaN,NaN,Cruz Fuentealba Eduardo Andres,Cruz,Fuentealba Eduardo Andres,NaN,121660733
4,340023981,1,Asegurados,13060950,3,0,13060950,3,13060950-3,1976-07-14,M,NaN,NaN,NaN,NaN,Prat Ayala Rodrigo Antonio,Prat,Ayala Rodrigo Antonio,rodrigopratayala@gmail.com,130609503


In [206]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
# Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_13456\192164855.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_13456\192164855.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [207]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(71, 30)

In [208]:
Pacientes_con_id.head()

,poliza,GRUPO,NOMBREGRUPO,RUTASEGURADO,DV,CARGA,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,nacimiento,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,340023981,1,Asegurados,9462157,7,0,9462157,7,9462157-7,1972-03-07,...,True,0018c00002nl8aYAAQ,94621577,Claudia Aranda Ortega,RUT,None,None,None,Mujer,22760503
1,340023981,1,Asegurados,9462157,7,2,20431786,0,20431786-0,2000-09-25,...,True,0018c00002nl8aWAAQ,204317860,M Ignacia Alvarez Aranda,RUT,None,None,None,Mujer,22760503
2,340023981,1,Asegurados,9462157,7,3,21182255,4,21182255-4,2003-09-08,...,True,0018c00002nl8aXAAQ,211822554,Antonia Alvarez Aranda,RUT,None,None,None,Mujer,22760503
3,340023981,1,Asegurados,12166073,3,0,12166073,3,12166073-3,1979-01-13,...,True,0018c00002ay6CCAAY,121660733,Eduardo Andres Cruz Fuentealba,RUT,None,None,None,Hombre,None
4,340023981,1,Asegurados,13060950,3,0,13060950,3,13060950-3,1976-07-14,...,True,0018c00002nl8puAAA,130609503,Rodrigo Antonio Prat Ayala,RUT,None,None,None,Hombre,None


In [209]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon":"Hombre"
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_13456\628618797.py:84: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")


,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,BillingStreet,BillingState,BillingCity,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [210]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
fecha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)



Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
#clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)


In [211]:
print(dia_hoy)

19


In [212]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
158865,a1Y8c00000C6VjSEAV,0018c00002amVQXAA2,a1W8c0000083iWsEAI,THB Seguros,0018c00002cXJU2AAO,Addval,Sin Holding Asociado,THB Seguros,Addval,Sin Póliza Asociada,None,EMP7483,A-0732
158866,a1Y8c00000C6VjTEAV,0018c00002amVQcAAM,a1W8c0000083iWsEAI,THB Seguros,0018c00002cXJU2AAO,Addval,Sin Holding Asociado,THB Seguros,Addval,Sin Póliza Asociada,None,EMP7483,A-0732
158867,a1Y8c00000C6VjVEAV,0018c00002amVkFAAU,a1W8c0000083iWsEAI,THB Seguros,0018c00002cXJU2AAO,Addval,Sin Holding Asociado,THB Seguros,Addval,Sin Póliza Asociada,None,EMP7483,A-0732
158868,a1Y8c00000C6VjXEAV,0018c00002amQHBAA2,a1W8c0000083iWsEAI,THB Seguros,0018c00002cXJU2AAO,Addval,Sin Holding Asociado,THB Seguros,Addval,Sin Póliza Asociada,None,EMP7483,A-0732
158869,a1Y8c00000C6VjaEAF,0018c00002amR2xAAE,a1W8c0000083idGEAQ,THB Seguros,0018c00002cXJaVAAW,Desert King Chile,Sin Holding Asociado,THB Seguros,Desert King Chile,Sin Póliza Asociada,None,EMP6850,A-1128


In [213]:
campaña_seleccionada.shape

(5554, 13)

In [214]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [215]:
if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [216]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)


In [217]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)